In [1]:
# Uploading libraries
import ast
import pandas as pd
import seaborn as sns
import numpy as np
from datasets import load_dataset
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from adjustText import adjust_text


# Uploading dataset
df_2019_q3 = pd.read_csv('Original_data_Q1_2019_Q1_2020/Divvy_Trips_2019_Q3.csv')


In [2]:
df_2019_q3.sample(10)

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
1402274,24976271,2019-09-15 21:03:05,2019-09-15 21:14:17,5326,672.0,86,Eckhart Park,71,Morgan St & Lake St,Subscriber,Male,1996.0
543715,24075570,2019-07-31 13:02:04,2019-07-31 13:07:47,1681,342.0,28,Larrabee St & Menomonee St,20,Sheffield Ave & Kingsbury St,Subscriber,Male,1977.0
368648,23891879,2019-07-22 14:43:09,2019-07-22 16:11:38,4036,"5,308.0",26,McClurg Ct & Illinois St,220,Clark St & Drummond Pl,Customer,NaN,NaN
103949,23599841,2019-07-06 20:47:12,2019-07-06 21:23:22,2648,"2,170.0",194,Wabash Ave & Wacker Pl,195,Columbus Dr & Randolph St,Customer,NaN,NaN
461198,23988976,2019-07-26 21:43:48,2019-07-26 22:06:31,5015,"1,363.0",117,Wilton Ave & Belmont Ave,114,Sheffield Ave & Waveland Ave,Customer,Female,1992.0
135768,23637946,2019-07-09 08:17:04,2019-07-09 08:20:39,5466,214.0,195,Columbus Dr & Randolph St,81,Daley Center Plaza,Subscriber,Male,1965.0
1626561,25208751,2019-09-30 09:08:34,2019-09-30 09:16:13,5993,458.0,140,Dearborn Pkwy & Delaware Pl,100,Orleans St & Merchandise Mart Plaza,Subscriber,Female,1987.0
535582,24067085,2019-07-30 21:25:31,2019-07-30 21:36:29,635,657.0,325,Clark St & Winnemac Ave,459,Lakefront Trail & Bryn Mawr Ave,Subscriber,Male,1981.0
1590970,25171655,2019-09-26 18:01:04,2019-09-26 18:21:47,813,"1,243.0",624,Dearborn St & Van Buren St,171,May St & Cullerton St,Subscriber,Male,1983.0
769107,24313565,2019-08-10 23:21:36,2019-08-10 23:35:57,4291,861.0,251,Clarendon Ave & Leland Ave,254,Pine Grove Ave & Irving Park Rd,Customer,Male,1993.0


In [4]:
df_2019_q3['usertype'] = df_2019_q3['usertype'].replace('Customer', 'casual')
df_2019_q3['usertype'] = df_2019_q3['usertype'].replace('Subscriber', 'member')

In [5]:
columns_to_drop = ["tripduration", "from_station_id", "from_station_name", "to_station_id", "to_station_name", "gender", "birthyear", "bikeid"]
df_clean_2019_q3 = df_2019_q3.drop(columns=columns_to_drop)

df_clean_2019_q3.sample(10)

,trip_id,start_time,end_time,usertype
392232,23916649,2019-07-23 17:01:07,2019-07-23 17:10:03,member
1184482,24749578,2019-09-03 08:31:31,2019-09-03 08:40:49,member
1384473,24957636,2019-09-14 17:32:49,2019-09-14 17:37:45,member
1091597,24652081,2019-08-28 22:21:16,2019-08-28 22:25:49,member
1003757,24560164,2019-08-24 04:28:36,2019-08-24 04:33:31,casual
1346756,24918296,2019-09-12 17:29:52,2019-09-12 17:41:49,member
562223,24094936,2019-08-01 09:40:08,2019-08-01 09:54:15,member
1552800,25132175,2019-09-24 17:46:57,2019-09-24 17:52:48,member
293704,23812911,2019-07-17 15:32:32,2019-07-17 15:37:08,member
485525,24014780,2019-07-28 10:15:48,2019-07-28 10:23:06,member


In [6]:
# Formatted columns start_time & end_time from Series to date&time then created new column where calculated trip duration

df_clean_2019_q3['start_time'] = pd.to_datetime(df_clean_2019_q3['start_time'], format='%Y-%m-%d %H:%M:%S')
df_clean_2019_q3['end_time'] = pd.to_datetime(df_clean_2019_q3['end_time'], format='%Y-%m-%d %H:%M:%S')


df_clean_2019_q3['ride_length_seconds'] = (df_clean_2019_q3['end_time'] - df_clean_2019_q3['start_time']).dt.total_seconds()

df_clean_2019_q3['ride_length_seconds'] = df_clean_2019_q3['ride_length_seconds'].astype(int)


def seconds_to_hhmmss(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


df_clean_2019_q3['ride_length'] = df_clean_2019_q3['ride_length_seconds'].apply(seconds_to_hhmmss)


df_clean_2019_q3.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length
1162540,24726534,2019-09-01 20:01:48,2019-09-01 21:13:07,casual,4279,01:11:19
223439,23738638,2019-07-13 16:37:19,2019-07-13 17:13:18,casual,2159,00:35:59
1149184,24712428,2019-09-01 10:25:08,2019-09-01 11:08:35,casual,2607,00:43:27
569914,24103012,2019-08-01 17:07:29,2019-08-01 17:51:09,member,2620,00:43:40
673209,24212948,2019-08-06 16:25:11,2019-08-06 16:46:14,casual,1263,00:21:03
90710,23584616,2019-07-06 11:19:13,2019-07-06 11:26:08,member,415,00:06:55
489316,24018803,2019-07-28 13:10:08,2019-07-28 14:25:29,casual,4521,01:15:21
325416,23846126,2019-07-19 16:32:17,2019-07-19 16:52:41,casual,1224,00:20:24
796716,24342643,2019-08-12 17:05:13,2019-08-12 17:14:24,casual,551,00:09:11
516570,24047262,2019-07-30 06:44:10,2019-07-30 06:53:02,member,532,00:08:52


In [7]:
df_clean_2019_q3['day_of_week'] = df_clean_2019_q3['start_time'].dt.day_name()
df_clean_2019_q3.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length,day_of_week
760637,24304632,2019-08-10 15:49:19,2019-08-10 16:13:47,casual,1468,00:24:28,Saturday
883560,24433737,2019-08-17 13:27:35,2019-08-17 13:41:26,member,831,00:13:51,Saturday
128799,23630732,2019-07-08 18:29:59,2019-07-08 18:51:11,casual,1272,00:21:12,Monday
398639,23923374,2019-07-23 20:09:43,2019-07-23 20:41:09,casual,1886,00:31:26,Tuesday
1409187,24983430,2019-09-16 12:29:37,2019-09-16 12:40:54,member,677,00:11:17,Monday
1556298,25135782,2019-09-24 19:50:48,2019-09-24 20:07:48,casual,1020,00:17:00,Tuesday
732309,24274632,2019-08-09 12:02:20,2019-08-09 12:08:02,member,342,00:05:42,Friday
422352,23948241,2019-07-25 07:00:17,2019-07-25 07:02:38,member,141,00:02:21,Thursday
819056,24366074,2019-08-14 00:46:52,2019-08-14 00:50:31,member,219,00:03:39,Wednesday
878986,24428908,2019-08-17 07:56:33,2019-08-17 08:00:59,member,266,00:04:26,Saturday


In [8]:
df_clean_2019_q3.to_csv('Cleaned_2019_Q3.csv', index=False)